# Target-Specific De Novo Peptide Binder Design with DiffPepBuilder

This notebook demonstrates how to use the [DiffPepBuilder](https://github.com/YuzheWangPKU/DiffPepBuilder) package to design peptides that bind to a target protein. We provide an example of the target ALK1 (Activin Receptor-like Kinase 1, PDB ID: [6SF1](https://www.rcsb.org/structure/6SF1)) to demonstrate the procedures of generating peptide binders.

## Setup

In [ ]:
#@title ### Download model assets
!git clone https://github.com/YuzheWangPKU/DiffPepBuilder.git
%cd DiffPepBuilder
!tar -xvf SSbuilder/SSBLIB.tar.gz -C SSbuilder
!wget https://zenodo.org/records/12794439/files/diffpepbuilder_v1.pth
!mkdir -p experiments/checkpoints/
!mv diffpepbuilder_v1.pth experiments/checkpoints/
%cd ..

In [ ]:
#@title ### Install dependencies
#@markdown Please restart the runtime as the warning suggests.
!pip install wget wandb fair-esm biotite pyrootutils easydict biopython tqdm ml-collections mdtraj GPUtil dm-tree tmtools
!git clone https://github.com/openmm/pdbfixer.git
%cd pdbfixer
!python setup.py install
%cd ..
!pip install hydra-core hydra-joblib-launcher

## Inference

In [ ]:
#@title ### Preprocess receptors
%cd DiffPepBuilder
!python experiments/process_receptor.py --pdb_dir examples/receptor_data --write_dir data/receptor_data --peptide_info_path examples/receptor_data/de_novo_cases.json

In [ ]:
#@title ### Run *de novo* generation
import os
os.environ['BASE_PATH'] = "/content/DiffPepBuilder"

!torchrun --nproc-per-node=1 experiments/run_inference.py \
  data.val_csv_path=data/receptor_data/metadata_test.csv \
  experiment.use_ddp=False \
  experiment.num_gpus=1 \
  inference.sampling.min_length=12 \
  inference.sampling.max_length=16

In [ ]:
#@title ## Evaluation
#@markdown **Note:** This may take ~15 min
!wget https://downloads.rosettacommons.org/downloads/academic/3.14/rosetta_bin_linux_3.14_bundle.tar.bz2
!tar -xvjf rosetta_bin_linux_3.14_bundle.tar.bz2
!rm -f rosetta_bin_linux_3.14_bundle.tar.bz2
!export ROSETTA_BIN_PATH="rosetta.binary.linux.release-371/main/source/bin"